# Gene essentiality

The authors perfomed a gene essentiality analysis to find which genes are important to still enable growth of the cells. Check how  they identified them.

Check gene knockouts on COBRApy docs to identify this important genes per cell line.

Compare your results. The authors compared their results with in-vitro data for validation (this would be very nice to do).